In [154]:
# TODO watch out with S=System / S=Subring notation

using Pkg
Pkg.activate("/Users/linussommer/signatures")

using AbstractAlgebra: derivative
using AlgebraicSolving

function natural(G)
    return [sigpair[2] for sigpair in G]
end;

function dehomogenize(S, R, var)
    return G
end

function rehomogenize(S, R, var)
    # TODO split this up
    S_new = []
    for s in S
        if parent(s) != R
            push!(S_new, s(var..., 1))
        else
            push!(S_new, s)
        end
    end
    S_new = identity.(S_new) # this is necessary because of the vector type...?
    return AlgebraicSolving._homogenize(S_new)
end;

function partial(q, ps)
    partial_q = 0
    for (i, pi) in enumerate(ps)
        partial_q =  partial_q + pi * derivative(q, i)
    end
    return partial_q
end;

function naive_sig_algorithm(qs, ps, R, var)
    S = AlgebraicSolving._homogenize(qs)
    g = qs
    G = sig_groebner_basis(S)
    while true
        g = [partial(gi, ps) for gi in g]
        g = [AlgebraicSolving.normal_form(gi, Ideal(natural(G))) for gi in g]
        if all(g .== 0)
            return G
        else
            append!(S, g)
            S = rehomogenize(S, R, var)
            G = sig_groebner_basis(S)
        end
    end    
    G = dehomogenize(G, R, var)
    return G
end;

  Activating project at `~/signatures`


In [182]:
R, (x1, x2, x3, x4, x5) = polynomial_ring(GF(65521),["x$i" for i in 1:5], ordering=:degrevlex)
ps = [x3, x4, x5*x1, x5*x2 - 1, 0] 
qs = [x1^2 + x2^2 - 1, x1]

G = naive_sig_algorithm(qs, ps, R, (x1, x2, x3, x4, x5))

In [181]:
G_p = natural(G)
G_pp = []
for g in G_p
    push!(G_pp, g(x1, x2, x3, x4, x5, 1))
end
G_pp = identity.(G_pp)

143-element Vector{fpMPolyRingElem}:
 x1^2 + x2^2 + 65520
 x1*x3 + x2*x4
 x4^2 + 65520*x5^2 + 65520*x2 + x5 + 1
 x2*x4*x5 + 32760*x3*x5^2 + 49140*x4
 x2^2*x3 + 65520*x1*x2*x4 + 65520*x3
 x2^2*x5^2 + 32760*x1*x5^3 + x5^3 + 49140*x2*x5 + 65520*x5^2 + 65520*x5 + 16381
 x3*x5^3 + 32760*x4*x5
 x4*x5^3
 x1*x5^4 + 32760*x2*x5^2 + 32761*x5
 x3*x4*x5^2 + 65519*x2*x5^3 + 65519*x2^2*x5 + 2*x2*x5^2 + 2*x2*x5 + 32762*x5^2 + 32762*x2 + 32759*x5 + 32759
 ⋮
 x3^3 + 43682*x3^2*x4 + 9*x2*x3*x5 + 65512*x1*x4*x5 + 21848*x3*x5^2 + 12*x4*x5^2 + 21856*x2*x3 + 65513*x1*x4 + 21855*x2*x4 + 21826*x3*x5 + 65507*x4*x5 + 43665*x3 + 65507*x4
 x2^4 + 32760*x1*x2^2*x5 + 2*x2^3 + 55283*x2*x3*x4 + 65520*x1*x2*x5 + 16378*x2^2*x5 + 4095*x3^2*x5 + 40952*x3*x4*x5 + 26620*x1*x5^2 + 49138*x2*x5^2 + 26618*x1*x2 + 32753*x2^2 + 40949*x3*x4 + 6144*x1*x5 + 8*x2*x5 + 12281*x5^2 + 38903*x1 + 28665*x2 + 36859*x5 + 20478
 x1*x2^3 + 65520*x1*x2^2*x5 + 49140*x5^4 + 16379*x1*x2^2 + 57335*x2^3 + 25082*x2*x3^2 + 4094*x2*x3*x4 + 53236*x1*x2

In [173]:
groebner_basis(Ideal(G_pp))

5-element Vector{fpMPolyRingElem}:
 x4
 x3
 x2 + 65520*x5
 x1
 x5^2 + 65520